In [7]:
from flask import Flask, render_template, request
import pickle
import numpy as np
import pandas as pd
import joblib


app = Flask(__name__)

model = joblib.load('custseg_pred_xgb.pkl')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        try:
            # Parse form data
            CustomerID = int(request.form['CustomerID'])
            StockCode = int(request.form['StockCode'])
            CountryCode = int(request.form['CountryCode'])
            UnitPrice = float(request.form['UnitPrice'])
            Quantity = int(request.form['Quantity'])
            Date = request.form['Date']

            # Create a DataFrame with the parsed data
            features = pd.DataFrame({
                'CustomerID': [CustomerID],
                'StockCode': [StockCode],
                'CountryCode': [CountryCode],
                'UnitPrice': [UnitPrice],
                'Quantity': [Quantity],
                'Date': [Date]
            })

            # Convert 'Date' column to datetime
            features['Date'] = pd.to_datetime(features['Date'], errors='coerce')

            # Create additional features
            features['Month'] = features['Date'].dt.month
            features['Year'] = features['Date'].dt.year
            features['Day'] = features['Date'].dt.day
            features['recency'] = (features['Date'] - features['Date'].min()).dt.days
            features['Frequency'] = features.groupby('CustomerID')['StockCode'].transform('count')
            features['TotalSpend'] = features['Quantity'] * features['UnitPrice']
            features['MonetaryValue'] = features.groupby('CustomerID')['TotalSpend'].transform('sum')

            # Drop unnecessary columns
            x = features.drop(columns=['Date', 'Day', 'Month', 'Year', 'CustomerID', 'StockCode'], axis=1)

            # Predict using the model
            output = model.predict(x)[0]

            if output == 0:
                flag = 'Low Value New Customer'
            elif output == 1:
                flag = 'High Value Frequent Customer'
            else:
                flag = 'High Value New Customer'

            outputline = 'The customer is a {}'.format(flag)
            return render_template('result.html', Prediction=outputline)

        except Exception as e:
            error_message = f"An error occurred: {str(e)}"
            return render_template('index.html', error=error_message)
    else:
        return render_template('index.html')


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Mar/2024 10:07:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2024 10:08:08] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2024 10:08:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2024 10:24:06] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [07/Mar/2024 10:24:31] "GET / HTTP/1.1" 200 -
